In [6]:
import pandas as pd
from db import call_db_json
sql = """ 
    SELECT
        mo.product_family,
        mo.product_model,
        mo.product_variant,
        mo.item_description,
        mo.order_quantity,
        mo.order_number,
        mo.product_length,
        mo.reference_number,
        mol.time_out,
        mol.facility_id        
    FROM manufacturing_orders mo
    JOIN manufacturing_order_logs mol ON mol.order_number = mo.order_number
    WHERE mo.order_status = '55'
        AND mo.order_release_code = 5
        AND mol.time_out >= '2022-01-01'::date
        AND mol.time_out <= '2022-12-31'::date
        AND mo.product_model IS NOT NULL
        AND mo.product_family IS NOT NULL
        AND mo.product_length IS NOT NULL
        AND mo.reference_number LIKE 'HK%'
        AND mo.item_description LIKE '%RAIL%'
        AND mo.product_family != 'TS'
    ORDER BY mo.order_number;
    """
base = pd.DataFrame(call_db_json(sql))

In [ ]:
# df = base[base['item_description'].apply(lambda x: x.find("K") != x.find("K)"))]
base['is_k'] = base['item_description'].apply(lambda x: x.find("K") != x.find("K)"))
# df = base[~base['is_k']]
df = base[base['facility_id'] == 'C0040']
rename = {'product_variant':'total'}
df['total'] = df['order_quantity'] * df['product_length'] / 1000
df['total'] = df['total'].astype(int)
df['type'] = df['product_family'] + df['product_model']
df = df.rename(columns=rename)
grouped = df.groupby(['type', 'is_k'])
summed = grouped['total'].sum()
summed = summed.reset_index() 
df = pd.DataFrame(summed)
df = df.pivot_table(index=['type'], columns='is_k', values=['total']).reset_index()
df.fillna(0)
df

In [ ]:
df.to_csv('ttt.csv')


In [ ]:
base['is_k'] = base['item_description'].apply(lambda x: x.find("K") != x.find("K)"))
df = base[~base['is_k']]
rename = {'product_variant':'total'}
df['total'] = df['order_quantity'] * df['product_length'] / 1000
df['total'] = df['total'].astype(int)
df['type'] = df['product_family'] + df['product_model']
df = df.rename(columns=rename)
grouped = df.groupby(['is_k', 'facility_id'])
summed = grouped['total'].sum()
summed = summed.reset_index() 
df = pd.DataFrame(summed)
df = df.pivot_table(index=['is_k'], columns='facility_id', values=['total']).reset_index()
df.fillna(0)

In [7]:
base

,product_family,product_model,product_variant,item_description,order_quantity,order_number,product_length,reference_number,time_out,facility_id
0,SRS-W,12,None,SRS12W-390LM RAIL,10.0,M000010,390.0,HK213868,2022-02-23 05:33:16-05:00,D0010
1,SRS-W,12,None,SRS12W-390LM RAIL,10.0,M000010,390.0,HK213868,2022-03-09 05:42:51-05:00,F0010
2,SRS-W,12,None,SRS12W-390LM RAIL,10.0,M000010,390.0,HK213868,2022-03-08 16:43:11-05:00,D0020
3,SRS-W,12,None,SRS12W-390LM RAIL,2.0,M000011,390.0,HK213869,2022-02-25 12:10:42-05:00,D0020
4,SRS-W,12,None,SRS12W-390LM RAIL,2.0,M000011,390.0,HK213869,2022-02-22 08:38:59-05:00,D0010
...,...,...,...,...,...,...,...,...,...,...
234960,SRS,9X,None,SRS9X-275LM RAIL,10.0,M999987,275.0,HK213846,2022-02-21 14:36:54-05:00,D0020
234961,SRS,9X,None,SRS9X-275LM RAIL,10.0,M999987,275.0,HK213846,2022-02-22 05:04:06-05:00,F0010
234962,SRS,9X,None,SRS9X-275LM RAIL,10.0,M999989,275.0,HK213847,2022-02-18 05:34:40-05:00,D0010
234963,SRS,9X,None,SRS9X-275LM RAIL,10.0,M999989,275.0,HK213847,2022-02-21 14:37:15-05:00,D0020
